In [2]:
import pandas as pd

df_crims = pd.read_csv('CMX_2024.csv')

In [ ]:
from datetime import datetime

# Convert 'fecha_hecho' to datetime
df_crims['fecha_hecho'] = pd.to_datetime(df_crims['fecha_hecho'])

df_last_year = df_crims[df_crims['fecha_hecho'].dt.year == 2024]
df_delict = df_last_year[df_last_year['categoria_delito'] != 'HECHO NO DELICTIVO']


#df_delict = df_delict[df_delict['categoria_delito'] != 'DELITO DE BAJO IMPACTO']



df_crimes = df_delict[['delito', 'longitud', 'latitud']]

display(df_crimes.shape)

#display(df_delict['categoria_delito'].value_counts())

df_xd = df_delict[df_delict['categoria_delito'] == 'DELITO DE BAJO IMPACTO']

display(df_xd['delito'].value_counts())

(119666, 3)

AttributeError: 'Series' object has no attribute 'distinct'

In [4]:
import geojson
from geojson import Feature, FeatureCollection, Point

def df_to_geojson(df):
    features = []
    for _, row in df.iterrows():
        if not pd.isna(row['latitud']) and not pd.isna(row['longitud']):
            point = Point((row['longitud'], row['latitud']))
            features.append(
                Feature(geometry=point, properties={"delito": row['delito']})
            )
    return FeatureCollection(features)

geojson_data = df_to_geojson(df_crimes)

# Save to a GeoJSON file
with open('crimes.geojson', 'w') as f:
    geojson.dump(geojson_data, f)

In [5]:
import folium

# Crear un mapa centrado en la latitud y longitud promedio
m = folium.Map(location=[df_crimes['latitud'].mean(), df_crimes['longitud'].mean()], zoom_start=12)

# Añadir el archivo GeoJSON al mapa
folium.GeoJson('crimes.geojson').add_to(m)

# Mostrar el mapa
m

In [4]:
import folium

def visualize_map(df):
    # Create a map centered around the average latitude and longitude
    m = folium.Map(location=[df['latitud'].mean(), df['longitud'].mean()], zoom_start=12)
    
    # Add points to the map
    for _, row in df.iterrows():
        if not pd.isna(row['latitud']) and not pd.isna(row['longitud']):
            folium.Marker(
                location=[row['latitud'], row['longitud']],
                popup=row['delito'],
                tooltip=row['categoria_delito']
            ).add_to(m)
    
    return m

# Example usage
map_visualization = visualize_map(df_crims)
map_visualization

KeyboardInterrupt: 